In [13]:
import numpy as np
import pandas as pd
from os import listdir
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error, \
recall_score, precision_score, accuracy_score, confusion_matrix
import math, random

In [ ]:
def HeaderMaker(PatientFile):
#     print(PatientFile)
    df_cleaned = pd.read_csv(PatientFile, header=None)

    df_cleaned = df_cleaned.rename(columns={0: "thorax_r_x", 1: "thorax_r_y", 2: "thorax_r_z"})
    df_cleaned = df_cleaned.rename(columns={3: "clavicula_r_x", 4: "clavicula_r_y", 5: "clavicula_r_z"})
    df_cleaned = df_cleaned.rename(columns={6: "scapula_r_x", 7: "scapula_r_y", 8: "scapula_r_z"})
    df_cleaned = df_cleaned.rename(columns={9: "humerus_r_x", 10: "humerus_r_y", 11: "humerus_r_z"})
    df_cleaned = df_cleaned.rename(columns={12: "ellebooghoek_r"})
    df_cleaned = df_cleaned.rename(columns={15: "thorax_l_x", 16: "thorax_l_y", 17: "thorax_l_z"})
    df_cleaned = df_cleaned.rename(columns={18: "clavicula_l_x", 19: "clavicula_l_y", 20: "clavicula_l_z"})
    df_cleaned = df_cleaned.rename(columns={21: "scapula_l_x", 22: "scapula_l_y", 23: "scapula_l_z"})
    df_cleaned = df_cleaned.rename(columns={24: "humerus_l_x", 25: "humerus_l_y", 26: "humerus_l_z"})
    df_cleaned = df_cleaned.rename(columns={27: "ellebooghoek_l"})
    
    return df_cleaned

def VisualizeItems(items, x, y, z):
    
    Dick_Classifier = {'cat': [],
                      'pat': [],
                      'meting': [],
                      'oef': [],
                       'split':[],
                      'Xenergie': [],
                      'Yenergie': [],
                      'Zenergie': []}
    
    for i in items:
        
        try:
            splitted = i.split('.')[0].split('_')
            Cat = int(splitted[0].split('/')[-1][3:])
            oef = splitted[1]
            meting = splitted[2]
            pat = int(splitted[3][3:]) + Cat * 1000
            split = splitted[4]
        except(IndexError):
            oef = 'failed'
        data = HeaderMaker(i)

        xlist = data[x]
        ylist = data[y]
        zlist = data[z]
        
        index = 0
        NewList = []
        # Versnelling berekenen
        for num, ColumnList in enumerate([xlist, ylist, zlist]):
            templist = []
            for index in range(1, len(ColumnList)):
                try:
                    vorige = ColumnList.iloc[index-1]
                    volgende = ColumnList.iloc[index+1]
                    templist.append(vorige - volgende)
                except(IndexError):
                    templist.append(0)
                    
            NewList.append({'x':list(ColumnList[1:]), 'y':templist})
                
        for axistype, item in enumerate(NewList):
#             fig = plt.figure()
#             ax = fig.gca()
#             for i in range(0, len(item['x']), 1):
#                 plt.plot(item['x'][i:i+2], item['y'][i:i+2], 'ro-')
#             cricle = plt.Circle((item['x'][0], item['y'][0]), 2, color='y')
#             ax.add_artist(cricle)
#             plt.title('cat: %s pat: %s oef: %s meting: %s' % (Cat, pat, oef, meting))
            
            TotaleOppervlakte = 0
            for index, CurrentXValue in enumerate(item['x']):
                if index == 0:
                    continue
                
                PreviousXValue = item['x'][index - 1]
                
                if CurrentXValue > PreviousXValue:
                    Multiplier = 1
                elif CurrentXValue < PreviousXValue:
                    Multiplier = -1
                else:
                    # Zelfde X waarde, dus geen oppervlakte
                    continue
                
                LowestValue = min(item['y'])
                CurrentyValue = item['y'][index]
                PreviousyValue = item['y'][index - 1]
                
                # Vierkant oppervlakte berekenen
                breedte = abs(abs(CurrentXValue) - abs(PreviousXValue))
                hoogte = abs(LowestValue) + abs(min([CurrentyValue, PreviousyValue]))
                Oppervlakte = breedte * hoogte
                
                # Driehoek oppervlakte berekeken
                hoogteDrie = abs(CurrentyValue - PreviousyValue)
                Oppervlakte = Oppervlakte + 0.5 * hoogteDrie * breedte
                
                TotaleOppervlakte += Oppervlakte * Multiplier
            
            if axistype == 0:
                Dick_Classifier['cat'].append(Cat)
                Dick_Classifier['pat'].append(pat)
                Dick_Classifier['meting'].append(meting)
                Dick_Classifier['oef'].append(oef)
                Dick_Classifier['split'].append(split)
                Dick_Classifier['Xenergie'].append(abs(TotaleOppervlakte))
            elif axistype == 1:
                Dick_Classifier['Yenergie'].append(abs(TotaleOppervlakte))
            elif axistype ==2:
                Dick_Classifier['Zenergie'].append(abs(TotaleOppervlakte))
            else:
                print('error')
    return Dick_Classifier
            

def CreateList(keywordslist):
    allitems = listdir('/datc/ortho/Cleaning/step2/')
    filteredlist = []
    for i in keywordslist:
        for x in allitems:
            if (i in x):
                filteredlist.append(x)
        allitems = filteredlist
        filteredlist = []
    finallist = []
    for i in allitems:
        finallist.append('/datc/ortho/Cleaning/step2/%s' % i)
    return finallist

In [ ]:
PatientFiles = CreateList(['meting'])
# print(PatientFiles)
# kasper = ['../EH1_cleaned.csv', 
#           '../RF1_cleaned.csv']
# x, y, z = ("thorax_r_x","thorax_r_y", "thorax_r_z")
#x, y, z = ("clavicula_r_x", "clavicula_r_y", "clavicula_r_z")
x, y, z = ("scapula_r_x", "scapula_r_y", "scapula_r_z")
#x, y, z = ("humerus_r_x", "humerus_r_y", "humerus_r_z")
# x, y, z = ("ellebooghoek_r", "ellebooghoek_l", "ellebooghoek_r")

# Print eerst de twee items van kasper, dan de gefilterde lijst
Dick = VisualizeItems(PatientFiles, x, y, z)

In [ ]:
df_dick = pd.DataFrame.from_dict(Dick)
df_dick

In [ ]:
def SplitData(df, xcol, ycol, percentage):
    allIndex = np.unique(df['pat'].tolist()) # Get all unique patients id's
    random.seed(2) # Set random seeed so the answer is the same
    
    PercentageIndex = []
    for i in np.unique(np.floor(allIndex / 1000)): # Cycle through each categorie (only cycles through the ones that are present)
        CatPatients = allIndex[allIndex<((i+1)*1000)] # Filters out patients who are bigger then our max
        CatPatients = CatPatients[CatPatients>=((i)*1000)] # Filters out patients that are smaller
        
        AmountItems = len(CatPatients)
        AmountRandom = math.floor(AmountItems*percentage)
        PercentageIndex.extend(random.sample(list(CatPatients), AmountRandom))        
        
    AmountItems = len(allIndex)
    AmountRandom = math.floor(AmountItems*percentage)
    
    PercentageIndex = random.sample(list(allIndex), AmountRandom)
    
    Percentagedf = df[xcol][df['pat'].isin(PercentageIndex)]
    Percentagey = df[ycol][df['pat'].isin(PercentageIndex)]
    
    Testdf = df[xcol][~df['pat'].isin(PercentageIndex)]
    Testy = df[ycol][~df['pat'].isin(PercentageIndex)]
    
    return (Percentagedf, Testdf, Percentagey, Testy)



In [ ]:
df_dick['bias'] = 1
df_dick['Tenergie'] = df_dick['Xenergie']+df_dick['Yenergie']+df_dick['Zenergie']

# X_train, X_test, y_train, y_test = train_test_split(df_dick[['bias','Xenergie','Yenergie','Zenergie','Tenergie']], df_dick['cat'], test_size = 0.2, random_state=42)
X_train, X_test, y_train, y_test = SplitData(df_dick, ['bias','Xenergie','Yenergie','Zenergie','Tenergie'], 'cat', 0.8)
lr = LogisticRegression()
lr.fit(X_train, y_train)

y_true = y_test
y_pred = lr.predict(X_test)

y_true = y_test
y_pred = lr.predict(X_test)

print(metrics.classification_report(y_true, y_pred,digits=3))